In [633]:
import pandas as pd
import numpy as np
from diskcache import Cache
from pathlib import Path
import pickle
import requests
import json
import os
import re

Definition et creation des fonctions pour l'embeddings a partir de l'API JDM

Mettre les generiques + les infos semantiques (relation 36) + marqueurs de type (numero de la relation)

Pour le vecteur B regarder l'article (determine ou non determine)

FAIRE DES PRODUITS SCALAIRES SUR DES VECTEURS NORMES


Useful links :

https://www.jeuxdemots.org/jdm-about.php

https://jdm-api.demo.lirmm.fr/schema

https://jdm-api.demo.lirmm.fr/v0/relations/from/{node1_id}

https://jdm-api.demo.lirmm.fr/v0/relations/to/{node2_name}

https://jdm-api.demo.lirmm.fr/v0/node_by_name/{node_name}

In [634]:
relations_path = Path(os.getcwd() + "/relations")

In [635]:
def load_relations(directory_path: str):

    rows = []
    num_to_rel = dict()

    for filename in os.listdir(directory_path):
        full_path = os.path.join(directory_path, filename)
        temp_data = re.split("[-.]",filename)
        relation_num = int(temp_data[0])
        relation_name = temp_data[1]
        num_to_rel[relation_num] = relation_name
        if os.path.isfile(full_path):
            
            with open(full_path, "r", encoding="utf-8") as f:
                data = f.read()
                data = re.split("\n",data)
                
                for l in data:
                     rows.append({"content": l, "sem_type": relation_num})

           

    df = pd.DataFrame(rows)

    print(df.head())
    

    return df, num_to_rel

In [636]:
def extraire_relation(phrase):
    # on retire Le, La, L’, Les, etc...
    phrase = phrase.strip()
    phrase = re.sub(r"^(l['’]|le|la|les|.)\s+", "", phrase, flags=re.IGNORECASE)
    
    # regex qui divise la phrase en 3 : le mot A, le connecteur et le mot B :
    pattern = r"^([\w\-éèêàùûôîç]+)\s+(d['’]|du|de la|de l’|de l'|de|des)\s+(.+)$"
   # print(phrase)
    m = re.match(pattern, phrase, flags=re.IGNORECASE)
    #print(m)
    if not m:
        return None
   
    A = m.group(1)
    connecteur = m.group(2)
    B = m.group(3).strip()
    
    return [A, connecteur, B]

In [637]:
# creation du dataset
df, relations_dict = load_relations(relations_path)
print(relations_dict)
temp_df = df['content'].apply(lambda x: extraire_relation(x))
temp_df = temp_df.dropna()
temp_df = temp_df.apply(lambda x: [x[0].lower(),x[1].lower(),re.sub(R"(^l’|\.$)",'',x[2]).lower()])

df['content'] = temp_df
df= df.dropna()
df
train_ds = []
for index, row in df.iterrows():
    words = row['content']
    train_ds.append((words[0],words[2],row['sem_type']))

print(train_ds[:20])

                     content  sem_type
0    Le travail de l’ouvrier         9
1   Le projet de l’ingénieur         9
2  Le discours du professeur         9
3     Le roman de l’écrivain         9
4    Le tableau de l’artiste         9
{9: ' Agent', 13: ' Conséquence', 5: ' Matière', 3: ' Holonymie', 15: ' Caractéristique', 11: ' Sujet', 16: ' Topic', 4: ' Quantification', 8: ' Lieu', 10: ' Auteur', 14: ' Possession', 2: ' Lien social', 1: ' Origine'}
travail de l’ouvrier
<re.Match object; span=(0, 20), match='travail de l’ouvrier'>
projet de l’ingénieur
<re.Match object; span=(0, 21), match='projet de l’ingénieur'>
discours du professeur
<re.Match object; span=(0, 22), match='discours du professeur'>
roman de l’écrivain
<re.Match object; span=(0, 19), match='roman de l’écrivain'>
tableau de l’artiste
<re.Match object; span=(0, 20), match='tableau de l’artiste'>
concert du musicien
<re.Match object; span=(0, 19), match='concert du musicien'>
découverte du chercheur
<re.Match object; span

In [638]:
embedding = dict()
embedding["info_sem"] = []
embedding["type_marqueur"] = set()
embedding["hyperonyme"] = []

In [639]:
# obtenir les relations d'un mot
node_name = "fromage"
res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/to/{node_name}")

In [640]:
# parser le json
res = json.loads(res.text)

In [641]:
"""types =   []
for x in res["relations"]:
    if(not x["type"] in types):
        #print(x)
        types.append(x["type"])

#cache.get(node_name)
types.sort()
print(types)"""
res

{'nodes': [{'id': 49, 'name': 'égrésoir', 'type': 1, 'w': 50},
  {'id': 326913823, 'name': 'tacos dorados', 'type': 1, 'w': 0},
  {'id': 326913824, 'name': 'tacos dorés', 'type': 1, 'w': 0},
  {'id': 875, 'name': 'agnella', 'type': 1, 'w': 50},
  {'id': 326913831, 'name': 'accorder les mets', 'type': 1, 'w': 27},
  {'id': 433903826,
   'name': '26f75feeed8933de154dad05c7c74c52',
   'type': 200,
   'w': 3},
  {'id': 326913833, 'name': 'préparations de taco', 'type': 1, 'w': 27},
  {'id': 2286, 'name': 'Colet', 'type': 1, 'w': 50},
  {'id': 2516, 'name': 'cambrouse', 'type': 1, 'w': 50},
  {'id': 2509, 'name': 'grasserie', 'type': 1, 'w': 50},
  {'id': 2781, 'name': 'Dabhia', 'type': 1, 'w': 50},
  {'id': 4668, 'name': 'ratinage', 'type': 1, 'w': 50},
  {'id': 250438638, 'name': 'Pâté de tête', 'type': 1, 'w': 25},
  {'id': 5764, 'name': 'fromage de cochon', 'type': 1, 'w': 50},
  {'id': 464090731,
   'name': 'lait au lait de bufflonne bio',
   'type': 1,
   'w': 0},
  {'id': 7189, 'name

In [642]:
# filtrer les noeuds pour les relations voulues + les poids negatifs
for r in res["relations"]:
    
    print(r)
    break
    if(r["w"] <= 0): continue
    
    if(r["type"] == 36):
        embedding["info_sem"].append(node["id"])
    elif(r["type"] == 6):
        embedding["hyperonyme"].append(node["id"])
    
    embedding["type_marqueur"].add(node["type"])

{'id': 61440, 'node1': 134936, 'node2': 141687, 'type': 0, 'w': 70.0}


In [643]:
embedding["hyperonyme"]

[]

In [644]:
embedding["type_marqueur"]

set()

In [645]:
embedding["info_sem"]

[]

In [672]:
# fonctions pour la similarite cosinus
def norm(v):
    res = 0
    for x in v:
        res += (x *x)
    return res**(1/2)

def dot(v1,v2):
    if(v1.shape != v2.shape):
        raise ArithmeticError
    res = 0
    for x,y in zip(v1,v2):
        res += x*y
    return res
        
def cosine_similarity(v1,v2):
    return dot(v1,v2)/(norm(v1) * norm(v2))

In [647]:
# classe qui cree la signature d'un terme et s'occupe de les cacher pr limiter le nombre de req
class SignatureLoader:
    def __init__(self,cache_dir="./jdm_cache"):
         self.cache = Cache(cache_dir)
            
        
    def get_signature(self, term):
        
        if term in self.cache:
            return self.cache[term]     
        sig = set()
        # peut-etre prends les relation /to/{term} ?
        res = requests.get(f"https://jdm-api.demo.lirmm.fr/v0/relations/from/{term}")
        print(res)
        if(res.status_code != 200):
            return sig
        try:
            res = json.loads(res.text)
        except:
            #print("Error decoding response")
            return sig
        
        try:
            for r in res["relations"]:

                if(r["w"] <= 0): continue

                if(r["type"] == 36):
                    sig.add(r["node2"])
                elif(r["type"] == 6):
                    sig.add(r["node2"])

                sig.add(r["type"])

                self.cache.set(term, sig, expire=None)
        except:
            return sig

        return sig


In [648]:
# Une regle est composee de 2 mots A et B, d'une relation et d'un "poids" (nombre de fusions)
class Rule:
    def __init__(self, sigA, sigB, relation, weight=1):
        self.sigA = sigA
        self.sigB = sigB
        self.relation = relation
        self.weight = weight

    def fuse(self, other):
        return Rule(
            set.union(self.sigA , other.sigA),
            set.union(self.sigB , other.sigB),
            self.relation,
            self.weight + other.weight
        )

In [649]:
# les signautres sont des vecteurs sparse avec des 0 et des 1
def signature_to_vector(sig, vocab):
    vec = np.zeros(len(vocab))
    for w in sig:
        vec[vocab[w]] = 1
    return vec

In [650]:
def learn_rules(examples, backend, threshold=0.5):
    # threshold definit la similarite minimale pour la fusion de 2 regles
    rules = []

    for A, B, rel in examples:
        sigA = backend.get_signature(A)
        sigB = backend.get_signature(B)

        new_rule = Rule(sigA, sigB, rel)

        fused = False
        for r in rules:
            if r.relation != rel:
                continue

            vocab = {w: i for i, w in enumerate(set.union(r.sigA , new_rule.sigA))}
            v1 = signature_to_vector(r.sigA, vocab)
            v2 = signature_to_vector(new_rule.sigA, vocab)
            if(len(v1) == 0): continue
            simA = cosine_similarity(v1,v2)
           # print(simA)

            vocab = {w: i for i, w in enumerate(set.union(r.sigB , new_rule.sigB))}
            v1 = signature_to_vector(r.sigB, vocab)
            if(len(v1) == 0): continue
            v2 = signature_to_vector(new_rule.sigB, vocab)
           # print(B,r.sigB)
           # print(v1.shape,v2.shape)
           # print(norm(v1),norm(v2))
           # print(dot(v1,v2))
           # t = dot(v1,v2)/(norm(v1)*norm(v2))
           # print(t)

            simB = cosine_similarity(v1,v2)

            if (simA + simB) / 2 >= threshold:
                merged = r.fuse(new_rule)
                rules.remove(r)
                rules.append(merged)
                fused = True
                break

        if not fused:
            rules.append(new_rule)

    return rules


In [651]:
def classify(A, B, backend, rules):
    best_score = -1
    best_rel = None
    # 1 - on calcul la signature des 2 termes
    sigA = backend.get_signature(A)
    sigB = backend.get_signature(B)

# 2 - on cherche la meilleure similarite cosinus entre toutes les regles de notre corpus
    for r in rules:
        # vocab = ensemble des donnees capturees dans les 2 signatures
        vocabA = {w: i for i, w in enumerate(set.union(sigA, r.sigA))}
        vecA = signature_to_vector(sigA, vocabA)
        vecRA = signature_to_vector(r.sigA, vocabA)
        simA = cosine_similarity(vecA,vecRA)
    


        vocabB = {w: i for i, w in enumerate(set.union(sigB, r.sigB))}
        vecB = signature_to_vector(sigB, vocabB)
        vecRB = signature_to_vector(r.sigB, vocabB)
        simB = cosine_similarity(vecB,vecRB)
        
        

        score = (simA + simB) / 2

        if score > best_score:
            best_score = score
            best_rel = r.relation

    return best_rel, best_score


In [652]:
sl = SignatureLoader()

In [653]:
# Entrainement
#training = train_ds[:100]
training = train_ds
rules = learn_rules(training, sl,threshold=1)

<Response [500]>


/tmp/ipykernel_194260/2677463472.py:16: RuntimeWarning: invalid value encountered in scalar divide
  return dot(v1,v2)/(norm(v1) * norm(v2))


<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500]>
<Response [500

In [654]:
# IMPORTANT SAUVEGARDER LES REGLES APPRISES
def save_rules(rules, filename="rules.pkl"):
    with open(filename, "wb") as f:
        pickle.dump(rules, f)

def load_rules(filename="rules.pkl"):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [655]:
save_rules(rules)

In [656]:
test = load_rules()

In [657]:
df

,content,sem_type
0,"[travail, de, ouvrier]",9
1,"[projet, de, ingénieur]",9
2,"[discours, du, professeur]",9
3,"[roman, de, écrivain]",9
4,"[tableau, de, artiste]",9
...,...,...
1874,"[vin, du, languedoc]",1
1875,"[laine, de, lama]",1
1876,"[mangue, de, colombie]",1
1877,"[condamnations, de, sarkozy]",1


In [674]:
# Inferences
A, conn, B = extraire_relation("Le chagrin du pere")
rel, score = classify(A, B, sl, rules)

print(relations_dict[rel], score)


chagrin du pere
<re.Match object; span=(0, 15), match='chagrin du pere'>
<Response [200]>
<Response [200]>
 Possession 0.7319275215078906


/tmp/ipykernel_194260/2677463472.py:16: RuntimeWarning: invalid value encountered in scalar divide
  return dot(v1,v2)/(norm(v1) * norm(v2))


In [659]:
print(relations_dict[df.iloc[276]["sem_type"]])

 Conséquence


In [660]:
len(sl.cache)
print(relations_dict)

{9: ' Agent', 13: ' Conséquence', 5: ' Matière', 3: ' Holonymie', 15: ' Caractéristique', 11: ' Sujet', 16: ' Topic', 4: ' Quantification', 8: ' Lieu', 10: ' Auteur', 14: ' Possession', 2: ' Lien social', 1: ' Origine'}


In [661]:
len(train_ds)

1433

In [662]:
for r in rules:
    if(r.weight > 1):
        print(r.weight)

3
2
4
2
3
3
5
6
3
5
3
3
3
4
3
3
3
3
9
4
4
6
4
6
6
4
4
3
4
4
4
4
4
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2


In [663]:
print(len(rules))

1281
